In [1]:
import json
import os
import random

import open3d as o3d

import torch
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
import sys 
sys.path.append('..')
import utils.misc as ws
import utils.data_utils
import utils.train_utils
import utils.eval_utils
import utils.mesh
import utils.dataset as d
import models.networks as arch


In [16]:
# DATA_SOURCE = '/home/ninad/Desktop/Docs/phd-res/proj-irvl-grasp-transfer/code/docker-data/output_dataset/'
# DATA_SOURCE = '/home/ninad/Desktop/multi-finger-grasping/output_dataset/'
DATA_SOURCE = '/mnt/Data/nak-8ycb-5grippers/'

object_model_name = "003_cracker_box_google_16k_textured_scale_1000"
# Load the object point cloud
model_pc_file = os.path.join(DATA_SOURCE, object_model_name, 'object_point_cloud.ply')
obj_pcd = o3d.io.read_point_cloud(model_pc_file)
gt_obj_pc_points = np.asarray(obj_pcd.points)
print(gt_obj_pc_points.shape)

# Load the normalization params:
norm_file = os.path.join(DATA_SOURCE, object_model_name, 'norm_params.npz')
norm_npz = np.load(norm_file)
offset = norm_npz['offset']
scale = norm_npz['scale']

# Normalize the object point cloud
gt_obj_pc_points = (gt_obj_pc_points - offset) / scale




(8194, 3)


In [4]:
EXPERIMENTS_DIR = '../experiments/all5_006_dsdf_50_varcmap'
CHECKPOINT = 'latest'
split_filename = os.path.join(EXPERIMENTS_DIR, 'split_train.json')
specs_filename = os.path.join(EXPERIMENTS_DIR, "specs.json")

LATENT_CODE_DIR = ws.latent_codes_subdir

In [5]:
specs = json.load(open(specs_filename))
latent_size = specs["CodeLength"]
gripper_weight = specs["GripperWeight"]
num_grippers = specs["NumGrippers"]

In [6]:
decoder = arch.dsdfDecoder(
    latent_size, 
    **specs["NetworkSpecs"]
    ).cuda()

decoder = torch.nn.DataParallel(decoder)

saved_model_state = torch.load(
    os.path.join(
        EXPERIMENTS_DIR, ws.model_params_subdir, CHECKPOINT + ".pth")
)

saved_model_epoch = saved_model_state["epoch"]

decoder.load_state_dict(saved_model_state["model_state_dict"])

decoder = decoder.module.cuda()

latent_vecs = ws.load_latent_vectors(EXPERIMENTS_DIR, CHECKPOINT)
print(latent_vecs.shape)

torch.Size([250, 128])


In [10]:
with open(split_filename, "r") as f:
    split = json.load(f)

# npz_filenames = utils.data_utils.dsdf_get_instance_filenames(
#     args.data_source, split)
cmap_f, grp_names, gpc_f, npz_filenames = utils.data_utils.get_instance_filelist(DATA_SOURCE, split)

# random.shuffle(npz_filenames) # WHY??? DISABLE THIS FOR CHECKING REPRODUCIBILITY

In [38]:
index_to_select = random.randint(0, len(npz_filenames)-1)

npz = npz_filenames[index_to_select]

full_filename = npz

print(index_to_select, npz[-35:])

latent_vec = latent_vecs[index_to_select].squeeze()
print(latent_vec.shape)


# Load the point cloud corresponding to this grasp/gripper
grp_pc_file = utils.eval_utils.extract_pcfile_from_npzfile(npz)
grp_pcd = o3d.io.read_point_cloud(grp_pc_file)
gt_grp_pc_points = np.asarray(grp_pcd.points)
# Normalize the loaded points
gt_grp_pc_points = (gt_grp_pc_points - offset) / scale

print(gt_grp_pc_points.shape)



149 0/sdf/HumanHand/sdf_graspnum_49.npz
torch.Size([128])
(271600, 3)


In [39]:
# with torch.no_grad():
#     queries, sdf_obj, sdf_grp = utils.eval_utils.eval_query_pc(decoder, latent_vec.cuda(), queries)

# Try with random queries instead:
with torch.no_grad():
    queries, sdf_obj, sdf_grp = utils.eval_utils.eval_random_query_pc(decoder, latent_vec.cuda(), num_samples=1000000)


In [40]:
EPS = 1e-4
gen_grp_points = queries[sdf_grp < EPS]
print(gen_grp_points.shape)

gen_obj_points = queries[sdf_obj < EPS]
print(gen_obj_points.shape)


torch.Size([21016, 3])
torch.Size([8297, 3])


In [41]:
utils.eval_utils.compute_pc_chamfer(gt_obj_pc_points, gen_obj_points)

(0.0020826099337984804, 0.0029303851513098698)

In [42]:
print(utils.eval_utils.compute_pc_chamfer(gt_grp_pc_points, gen_grp_points))

(0.007746310365695244, 0.00013065562264301015)
